In [ ]:
#!pip install qiskit.aqua
#!pip install qiskit==0.31.0
# AH: For some reason qiskit has dependency issues so these packages need to be installed

In [64]:
import numpy as np
import time

from qiskit import BasicAer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import SPSA, COBYLA
from qiskit.circuit.library import TwoLocal, ZZFeatureMap, EfficientSU2
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name

seed = 10599
aqua_globals.random_seed = seed

## Loading Data

In [5]:
data = np.load('data.npz')
sentence = data['sentence']
label = data['label']
n_train = 300
n_val = 200
n_test = 100
x_train, y_train = sentence[:n_train], label[:n_train]
x_val, y_val = sentence[n_train:n_val+n_train], label[n_train:n_val+n_train]
x_test, y_test = sentence[n_train+n_val:n_val+n_train+n_test], label[n_train+n_val:n_val+n_train+n_test]

In [30]:
feature_dim = len(x_train[0])
feature_dim

5

In [58]:
def split_to_labels(data, label):
    res = {}
    for i in range(len(label)):
        if label[i] not in res:
            res[label[i]] = []
        res[label[i]].append(data[i])
    
    return res

In [59]:
training_input = split_to_labels(x_train, y_train)
val_input = split_to_labels(x_val, y_val)

## Constructing Model

In [60]:
# Config
maximizing_steps = 4
encoding_repetition = 1
model_repetition = 1
simulator_shots = 1000

In [63]:
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=encoding_repetition)
optimizer = COBYLA(maxiter=maximizing_steps)
var_form = EfficientSU2(feature_dim, reps=model_repetition)
# TODO: append lambeq in front of vqc or replace feature_map
vqc = VQC(optimizer, feature_map, var_form, training_input, val_input)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=simulator_shots, seed_simulator=seed, seed_transpiler=seed)

In [72]:
t0 = time.time()

# Running the model
vqc.run(quantum_instance)

{'num_optimizer_evals': 4,
 'min_val': 1.3149781021939224,
 'opt_params': array([ 0.56818953,  2.69546501, -0.36402734, -1.31624338,  0.41340523,
        -1.01974468, -0.24092391,  1.71842934,  0.26132169, -1.11700633,
         1.33113277, -1.26841527,  0.59526398,  2.39563254,  1.21393098,
         1.3333343 , -0.34251393, -1.95753451, -0.49437309, -1.79217551]),
 'eval_time': 28.44213366508484,
 'eval_count': 4,
 'training_loss': 1.3149781021939224,
 'testing_accuracy': 0.105,
 'test_success_ratio': 0.105,
 'testing_loss': 1.3796076479690071}

In [73]:
t1 = time.time()
print(f">>> Task finished in {abs(t1 - t0)/60:.2f} min")

>>> Task finished in 0.55 min
